In [1]:
import numpy as np
import pandas as pd
import yaml
from pathlib import Path
from tqdm.notebook import tqdm
import re

from gigachat_synthetic_news.llm import llm_output, gigachat_stub


Set global variables and read configuration data

In [ ]:
## global variables
config_path = Path('../configs/config.yml')
config_dir = config_path.parent

# configs for the model
with config_path.open('r', encoding='utf-8') as fh:
    read_config = yaml.load(fh, Loader=yaml.FullLoader)
creds = read_config['creds']
model = read_config['model']

# prompt for request
system_prompt_path = config_dir / read_config['system_prompt_path']
with system_prompt_path.open('r', encoding='utf8') as file_obj:
    system_prompt_template = file_obj.read()

# prompt for request
prompt_path = config_dir / read_config['prompt_path']
with prompt_path.open('r', encoding='utf8') as prompt_file:
    prompt_template = prompt_file.read()


In [ ]:
# our dataset
df_path = Path('../data/processed/synthetic_news.xlsx')
df = pd.read_excel(df_path)
df = df.loc[:, ~df.columns.astype(str).str.startswith('Unnamed:')]
df


,type,No,text,impact
0,Отчет о финансовых результатах,1,Чистая прибыль Газпрома в I квартале увеличила...,1.0
1,Отчет о финансовых результатах,2,Выручка Лукойла в II квартале достигла рекордн...,1.0
2,Отчет о финансовых результатах,3,Норникель объявил о значительном росте прибыли...,1.0
3,Отчет о финансовых результатах,4,Яндекс опубликовал отчет о финансовых результа...,1.0
4,Отчет о финансовых результатах,5,Роснефть сообщила о росте добычи нефти на 4% в...,1.0
...,...,...,...,...
615,Инвестиции в капитал,16,Черкизово вложит $100 млн в расширение своего...,0.8
616,Инвестиции в капитал,17,Русагро инвестирует $150 млн в строительство ...,0.8
617,Инвестиции в капитал,18,ФосАгро планирует построить новый горно-обога...,0.8
618,Инвестиции в капитал,19,Мечел вложит $100 млн в разработку Эльгинског...,0.8


In [ ]:
# Let's generate 3 additional versions for each synthetic news item: a short one, a medium one, and a long one.
all_prompts = {}

for i in range(1, df.shape[0]+1):
    all_prompts[f'{i}_short'] = system_prompt_template.format(length_bounds = '50-70')
    all_prompts[f'{i}_medium'] = system_prompt_template.format(length_bounds = '80-110')
    all_prompts[f'{i}_long'] = system_prompt_template.format(length_bounds = '120-150')

# news dictionary
news_dictionary = {f'{i+1}': [news] for i, news in enumerate(df['text'])}
# add impact of generated news
k = 0
for i in list(news_dictionary.keys()):
    impact = df.loc[k,'impact']
    news_dictionary[i].append(impact)
    k = k+1

Заглушка для гигачата - с первого раза он никогда не работает, нужно один раз его запустить через try-except

In [ ]:
# Function to extract the number before the '_' character
def extract_number(s):
    match = re.match(r'(\d+)', s)
    return match.group(1) if match else s

In [6]:
gigachat_stub(creds, model)()

c:\Programming\venvs\telegram_news\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Giga generation stopped with reason: length


Росбанк является одним из крупнейших банков в России,


'Росбанк является одним из крупнейших банков в России,'

In [ ]:
# create final df
final_df = pd.DataFrame({'No':[], 'text':[], 'is_synthetic':[], 'impact':[]})
i = 1
k =0
for prompt in tqdm(list(all_prompts.keys())): # [:12]
    k = k+1
    news = news_dictionary[extract_number(prompt)][0]
    impact = news_dictionary[extract_number(prompt)][1]
    # add the original news item
    if i==1:
        curr_df = pd.DataFrame({'No':[1], 'text':[news], 'is_synthetic':[1], 'impact':[impact]})
        final_df = pd.concat([final_df, curr_df])
        i = 2
    else:
        continue

    # Generate news
    try:
        output = llm_output(news, 
                            all_prompts[prompt],
                            prompt_template, 
                            temperature = 1.5)     
    except:
        print(f'{k}_{news}')
        output = None
    
    curr_df = pd.DataFrame({'No':[i], 'text':[output], 'is_synthetic':[0], 'impact':[impact]})
    if i==4:
        i = 1
    else:
        i += 1

    final_df = pd.concat([final_df, curr_df])

  0%|          | 0/1860 [00:00<?, ?it/s]

In [17]:
final_df.reset_index(drop=True).loc[7,'text']

'Во втором квартале текущего года выручка компании "Лукойл" достигла исторического максимума. Этот успех обусловлен двумя ключевыми факторами: повышением стоимости нефти на мировом рынке и увеличением объемов реализации продукции. Оба этих фактора оказали положительное влияние на финансовые показатели компании, что свидетельствует о ее стабильности и успешности в текущих экономических условиях.'

Add news type

In [ ]:
# Add a new column
new_column_type = [[i]*20*4 for i in range(1,32)]
new_column_type = list(np.array(new_column_type).flatten())

final_df_copy = final_df.copy().reset_index(drop=True)
final_df_copy['news_type'] = new_column_type

In [ ]:
# Save
final_df_copy.to_excel(Path('../data/processed/generated_news.xlsx'), index=False)
